## 5
### (a)

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
## set data
N = 3
T = 4
K = 100
E = 2
C_i = [30, 40, 50]
H_i = [6, 8, 10]
I_i = [50, 40, 20]
D_it = [
    [60, 30, 30, 40],
    [30, 15, 35, 80],
    [20, 35, 30, 30]
]

In [3]:
#create model
model = gp.Model("Cake")

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [4]:
#create variable
x = model.addVars(N, T, T,vtype=GRB.CONTINUOUS, name="x_ijt")
x

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 3): <gurobi.Var *Awaiting Model Update*>,


In [5]:
#set objective function
expr = gp.quicksum([C_i[i] * gp.quicksum([ gp.quicksum([x[i,j,k] for j in range(T)]) for k in range(T)]) for i in range(N)])
expr += gp.quicksum([
        gp.quicksum([
            H_i[i] * (I_i[i] + 
                gp.quicksum([
                    gp.quicksum([
                    x[i,j,k] 
                    for k in range(j, T)])
                    - D_it[i][j] 
                for j in range(t+1)]))
        for i in range(N)]) 
        for t in range(T)])


In [6]:
model.setObjective(expr, GRB.MINIMIZE)

In [7]:
# Add constrains
model.addConstrs((gp.quicksum([gp.quicksum([x[i,j,k] for k in range(j, T)]) for i in range(N)]) <= K for j in range(T)), name="produce at most K cake per day")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [8]:
model.addConstrs((
    gp.quicksum([ gp.quicksum([x[i,j,k] for j in range(k+1)]) for k in range(t+1)]) >= gp.quicksum([D_it[i][k] for k in range(t+1)]) - I_i[i]
        for t in range(T)
        for i in range(N)
))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>}

In [9]:
model.addConstr(
    gp.quicksum([
       gp.quicksum([
           gp.quicksum([
               x[i,j,k] for k in range(j+E, T)
           ]) for j in range(T-E)
       ]) for i in range(N)
    ])
    <= 0
)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.addConstrs(
    gp.quicksum([
        x[i,j,t] for j in range(t+1)
    ])
    >= D_it[i][t]
    for t in range(E-1, T)
    for i in range(N)
)

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>}

In [11]:
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 26 rows, 48 columns and 126 nonzeros
Model fingerprint: 0x661f6c52
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve removed 4 rows and 27 columns
Presolve time: 0.01s
Presolved: 22 rows, 21 columns, 84 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.5900000e+03   9.437500e+01   0.000000e+00      0s
      17    1.4340000e+04   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.02 seconds
Optimal objective  1.434000000e+04


In [13]:
for v in model.getVars():
    if(v.x != 0):
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % model.objVal)

x_ijt[0,0,0] 10
x_ijt[0,0,1] 25
x_ijt[0,1,1] 5
x_ijt[0,1,2] 30
x_ijt[0,2,3] 40
x_ijt[1,1,1] 15
x_ijt[1,1,2] 15
x_ijt[1,2,2] 20
x_ijt[1,2,3] 10
x_ijt[1,3,3] 70
x_ijt[2,1,1] 35
x_ijt[2,2,2] 30
x_ijt[2,3,3] 30
Obj: 14340


## (b)